# Exercice 2 : Augmentation de la capacité d'un modèle

In [ ]:
%matplotlib notebook
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

Choix de la fonction à apprendre pour l'exemple : ici un polynome de degré 5.

In [ ]:
listeParamPoly = [0.03, 0.2, -1, -10, 100]

In [ ]:
def generate_data(N):
    x = np.random.uniform(-10,10,N)
    y = np.polyval(listeParamPoly,x) + np.random.normal(0.0, 15.0, N)
    return x.reshape(-1, 1), y

Génération des données : Choisir le nombre de points à utiliser :

In [ ]:
dataPoints = 20

In [ ]:
X,y = generate_data(dataPoints)

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, y, 'o')
ax.plot(np.linspace(-10,10,100), np.polyval(listeParamPoly,np.linspace(-10,10,100)), color='black', linewidth=3)
ax.set_title('Data set')
ax.set_ylabel('y')
ax.set_xlabel('x')
plt.show()

# Définition des fonctions de caractéristiques

Ajoutez dans le vecteur $\varphi$ autant de fonctions caractéristiques que vous le souhaitez. Idéalement non-linéaires.

In [ ]:
phi = [
       lambda x: x**0, 
       lambda x: x**1,
       lambda x: x**2,
       lambda x: x**3,
       lambda x: np.abs(x),
       lambda x: x > -5.0,
       lambda x: x > 0.0,
       lambda x: x > 5.0,
       lambda x: np.cos(0.1*x),
       lambda x: np.cos(x),
       lambda x: np.sin(0.1*x),
       lambda x: np.sin(x),
      ]

On va utiliser ce vecteur pour faire la projection. Voici la fonction de projection:

In [ ]:
def feature_space_projection(X, phi):
    X_features = []
    for i in range(0, len(phi)):
        X_features.append(np.apply_along_axis(phi[i], 0, X))
    X_augmented = np.concatenate(X_features, axis=1)
    return X_augmented

# Calcul de la projection dans l'espace des caractéristiques

Calcul de la projection puis affichage de notre entrée dans le nouvel espace.

In [ ]:
X_augmented = feature_space_projection(X, phi)

In [ ]:
X_augmented[0,:]

# Entraînement du modèle

Maintenant qu'on a ouvert la boîte à l'exercice précédent, on va utiliser le modèle de regression linéaire [`linear_model.LinearRegression()`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) de `scikit-learn`. On pourrait utiliser d'autres modèles [linéaires](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model).

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X_augmented, y)

# Calcul de l'erreur sur l'ensemble d'entraînement vs le "vrai" risque

In [ ]:
from sklearn.metrics import mean_squared_error

### Erreur d'entraînement

Calcul de la fonction de perte [MSE](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) sur les données d'entrainement:

In [ ]:
y_pred = reg.predict(X_augmented)
training_error = mean_squared_error(y, y_pred)
print("L'erreur d'entraînement du modèle appris est : %5.2f" % training_error)

### Estimation du vrai risque

Et si on valide (non réalistiquement) sur la *vraie* distribution des données:

In [ ]:
X_test,y_test = generate_data(10000000)
X_test_augmented = feature_space_projection(X_test, phi)
y_test_pred = reg.predict(X_test_augmented)
test_error = mean_squared_error(y_test, y_test_pred)
print("Vrai risque du modèle appris est : %5.2f" % test_error)

# Attention au sur-apprentissage

Lorsque le nombre de caractéristiques utilisé est très important, la capacité augmente énormément jusqu'à apprendre *par coeur* les données.

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, y, 'o')
ax.set_title('Polynôme')
ax.set_ylabel('y')
ax.set_xlabel('x')

linspace_x = np.linspace(-10, 10, num=100000)
linspace_x = np.expand_dims(linspace_x, axis=1)

linspace_X_augmented = feature_space_projection(linspace_x, phi)

y_pred = reg.predict(linspace_X_augmented)
ax.plot(linspace_x, y_pred, color='red', linewidth=3)

plt.show()

# Régularisation et hyperparamètres

Il nous faut alors soit réduire le nombres de caractéristiques (et donc la dimensionnalité de l'espace de projection), soit *régulariser* nos paramètres pour assurer une bonne *généralisation*.

On va utiliser ici [ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet) de `scikit-learn` qui permet de combiner les normes $L_1$ et $L_2$ pour la régularisation.

Vous pouvez chosir ici le taux de régularisation et la proportion de régularisation $L_1$ vs $L_2$.

In [ ]:
taux_de_regularisation = 0.001
ratio_normes = 0.5

On définit ensuite le modèle et on l'entraine.

In [ ]:
reg2 = linear_model.ElasticNet(alpha=taux_de_regularisation, copy_X=True, fit_intercept=True, l1_ratio=ratio_normes,
      max_iter=10000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='random', tol=0.0001, warm_start=False)
reg2.fit(X_augmented, y)

Calcul du risque empirique : 

In [65]:
y_pred = reg2.predict(X_augmented)
training_error = mean_squared_error(y, y_pred)
print("L'erreur d'entraînement du modèle appris est : %5.2f" % training_error)

L'erreur d'entraînement du modèle appris est : 144.59


Calcul du *vrai* risque:

In [ ]:
X_test,y_test = generate_data(1000000)
X_test_augmented = feature_space_projection(X_test, phi)
y_test_pred = reg2.predict(X_test_augmented)
test_error = mean_squared_error(y_test, y_test_pred)
print("Vrai risque du modèle appris est : %5.2f" % test_error)

Affichage du modèle régularisé :

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, y, 'o')
ax.set_title('Polynôme')
ax.set_ylabel('y')
ax.set_xlabel('x')

linspace_x = np.linspace(-10, 10, num=100000)
linspace_x = np.expand_dims(linspace_x, axis=1)

linspace_X_augmented = feature_space_projection(linspace_x, phi)

y_pred = reg2.predict(linspace_X_augmented)
ax.plot(linspace_x, y_pred, color='red', linewidth=3)

plt.show()

Choix fait des paramètres de régularisation par ElasticNet:

In [ ]:
print(reg2.coef_)